TODO in class
* install BeautifulSoup if you do not have it already
* pip install beautifulsoup4 
* note the 4 at the end
* find two URLs that you want to scrape on SS.com
* scrape the data from the first page
* scrape the data from the second page
* you can use the functions I provided in the class

* save both as csv files
* and make plot of the data - up to you what you want to plot


* homework will be to try to srape date from some other site - such as wikipedia

In [1]:
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
url1 = "https://www.ss.com/en/animals/dogs/"
url1

'https://www.ss.com/en/animals/dogs/'

In [4]:
url2 = "https://www.ss.com/en/electronics/computers/pc/"
url2

'https://www.ss.com/en/electronics/computers/pc/'

In [39]:
req = requests.get("https://www.ss.com/en/animals/dogs/") # so here we make a call to webpage via HTTP GET request and get something back
req.status_code # we could add if to check for 200

200

In [40]:
soup = BeautifulSoup(req.text, 'lxml') 
soup.title

<title>SS.COM Animals - Dogs, puppies, Prices - Advertisements</title>

In [41]:
# lets combine the above cells into a function which will always get us columns
def getColList(soup):
    column_list = ["description","url"] # we decided to that we need these two column names no matter the html
    headline = soup.find("tr", {"id":"head_line"})
    headtds = headline.find_all("td")
    headcolumns = [el.text for el in headtds[1:]] # this will get all column names starting with 2nd in HTML
    column_list += headcolumns
    return column_list

In [42]:
column_names = getColList(soup)
column_names

['description', 'url', 'Breed', 'Age', 'Price']

In [43]:
def getRowList(soup):
    trows = soup.find_all('tr')
    aprows = [row for row in trows if row.get('id',"").startswith("tr_") and not row.get('id',"").startswith("tr_bnr") ]
    return aprows

In [45]:
def getRow(row,colist=column_names):
    row_tds = row.find_all('td')
    rowDict = {}
    if len(row_tds) < 3: # a little sanity check
        print("Hmm bad row")
        return rowDict
    
    rowDict[colist[0]] = row_tds[2].text # so the big assumption is that we always get description in 3rd column
    rowDict[colist[1]] = "https://ss.com" + row_tds[1].find('a').get('href')
    for td,key in zip(row_tds[3:],colist[2:]): 
        rowDict[key] = td.text
    return rowDict

In [46]:
def getRows(rowlist,colist=column_names):
    return [getRow(row, colist=colist) for row in rowlist] # so return a list of dictionaries

In [47]:
# so with this function I can get full dataframe from a single page on ss.com not only apartments
def getDFfromURL(url):
    # print("getting data from", url)
    req = requests.get(url)
    if req.status_code != 200:
        print("Request Fail with", req.status_code)
        return None # maybe return empty dataframe here
    soup = BeautifulSoup(req.text, 'lxml')
    column_names = getColList(url)
    rowlist = getRowList(url)
    rows = getRows(rowlist,colist=column_names)
    return pd.DataFrame(rows, columns=column_names)

In [50]:
apt_rows[0] # first row of ads

<tr id="tr_52255531"><td class="msga2 pp0"><input id="c52255531" name="mid[]" type="checkbox" value="52255531_14023_0"/></td><td class="msga2"><a href="/msg/en/animals/dogs/outbred/apxif.html" id="im52255531"><img alt="" class="isfoto foto_list" src="https://i.ss.com/gallery/5/997/249152/49830343.th2.jpg"/></a></td><td class="msg2"><div class="d1"><a class="am" data="JTk5ayU5QiU4MCU4Q2olREZpJTlBJTdCJTg5aSU5Q3ElOTYlODAlODVrJTk0byU5NHolODNmJTkz|c8dGS6" href="/msg/en/animals/dogs/outbred/apxif.html" id="dm_52255531">Mamma Vācu aitu sugas, tēvs Haski. 
Labs sargs jūsu mājai un uzticams draugs. 
6 meite</a></div><div class="ads_region">Ogre and reg.</div></td><td c="1" class="msga2-o pp6" nowrap="">Outbred</td><td c="1" class="msga2-o pp6" nowrap="">1.1 mon.</td><td c="1" class="msga2-o pp6" nowrap="">50  €</td></tr>

In [52]:
row_ads = getRows(apt_rows)
row_ads[-3:] # last 3 ads

[{'description': 'Предлагаются к резервированию щеночки мальтипу- все мальчики. Мама- мальтийская болонка,Jekabpils and reg.',
  'url': 'https://ss.com/msg/en/animals/dogs/caniche-miniature/cxplgm.html',
  'Breed': 'Caniche Miniature',
  'Age': '0.5 mon.',
  'Price': '950  €'},
 {'description': 'Silikona birste suņu un kaķu mazgāšanai dzeltena (5626_1)\r\n\r\nProdukta īpašības:\r\n IzgataRiga',
  'url': 'https://ss.com/msg/en/animals/dogs/accessories/cxlkb.html',
  'Breed': '-',
  'Age': '-',
  'Price': '3.50  €'},
 {'description': 'Предлагается мальчик йоркширского терьера, малыш активный, имеет евро паспорт, прививку Riga district',
  'url': 'https://ss.com/msg/en/animals/dogs/yorkshire-terrier/emkfn.html',
  'Breed': 'Yorkshire Terrier',
  'Age': '2 mon.',
  'Price': '600  €'}]

In [53]:
dtemp = pd.DataFrame(row_ads, columns=column_names)
dtemp.shape

(30, 5)

In [54]:
dtemp.head()

,description,url,Breed,Age,Price
0,"Mamma Vācu aitu sugas, tēvs Haski. \r\nLabs sa...",https://ss.com/msg/en/animals/dogs/outbred/apx...,Outbred,1.1 mon.,50 €
1,"Весёлый, ласковый мальчик ждёт любящую семью. ...",https://ss.com/msg/en/animals/dogs/caniche-min...,Caniche Miniature,3 mon.,700 €
2,Pēc mūsu 13 gadus vecā Labradora zaudēšana pag...,https://ss.com/msg/en/animals/dogs/labrador-re...,Labrador Retriever,3 mon.,buy
3,"Nodosim mīļās, gādīgās, labās rokās mazos suņu...",https://ss.com/msg/en/animals/dogs/labrador-re...,Labrador Retriever,2 mon.,100 €
4,"Очаровательные, любознательные малыши ищут отв...",https://ss.com/msg/en/animals/dogs/outbred/cbe...,Outbred,3 mon.,-


In [56]:
dtemp.to_excel("dogs_data.xlsx")

In [58]:
imanta = "https://www.ss.com/en/animals/dogs/"
idf = getDFfromURL(imanta)
idf.head()

TypeError: slice indices must be integers or None or have an __index__ method